## Antibiogram Report Exploration and Data Report

Identification (ID) and Antibiotics Susceptibility Testing (AST) were perform with Phoenix BD System and data were taken from Epicenter BD Software, related data were from January - March 2013 From Dr. Soetomo Hospital 

In this report we try to explore data, and categorize and grouping each columns

There are 14 columns
1. Patient ID : Patient Hospital Identification Number
2. Patient Name : Patient Name
3. Accession ID : Laboratory Number
4. Date
5. Morphology of Bacteria : Gram + Bacilli, Gram + Cocci, Gram - Bacilli
6. Genus of Bacteria
7. Names of Bacteria
8. Antibiotic Class
9. Antibiotic Name
10. Interpretation Result
    * 'S', 'R', 'X', nan, 'I', 'N'
11. MIC Breakpoint
12. Resistance Marker
13. Wards Name : wards in Soetomo District Hospital, this will group according to each Departement
14. Specimen Type


In [1]:
import pandas as pd
import os

file = 'AST_jan_march_2013.csv'
columns_fields = ['Patient_ID', 'Patient_Name','Accesion_ID','Date',
                  'AB_Class','Organism_ID','AB_Name','Morphology_ID',
                  'Genus_ID','SIR','MIC','RM','Month','Ward_ID','SpecType']
df = pd.read_csv(file, delimiter=',', header=None, names=columns_fields)
df.describe()

,Patient_ID,Patient_Name,Accesion_ID,Date,AB_Class,Organism_ID,AB_Name,Morphology_ID,Genus_ID,SIR,MIC,RM,Month,Ward_ID,SpecType
count,79110,78778,81119,81119,81119,81119,81119,81119,81119,74367,74486,53180,81119,81119,81119
unique,1784,1689,3121,877,11,137,34,4,44,5,45,11,3,73,45
top,LE46628,INAYATUL MUSTAQH,1703161758C,3/9/2013 10:02,Beta-lactams,Escherichia coli,Ampicillin,Gram - Bacilli,Staphylococcus,R,>16,Extended Spectrum Beta-lactamase,2013/03,IRJ / Instalasi Rawat Jalan,Urine
freq,624,624,105,503,39019,14995,3986,45398,29801,40524,9187,13874,30868,5886,13748


## 1st STEP --> Clean The Data

* There's a 3121 Accesion ID --> this is data that we want to process
* We need to drop columns blank data in Patient_ID, Patient_Name, Accesion_ID
* we need to drop some result with blank data in Morphology ID, Genus_ID, Organism_ID
* we drop some Unspecified Result
* Create Antibiotic Abbrevation for more convinient way

### RESULT
* We Clean Up 184 specimen
* There's a :
    * 2937 specimen with full field
    * 3 Morphology
    * 132 Organism Name
    * 42 Genus Name
    * 11 Antibiotic Class
    * 3 Antibiotic Result = S, I, R
    * 11 Resistance Marker
    * 73 Wards
    * 42 Specimen Type

In [2]:
def clean(d_frame):
    df_work = d_frame[d_frame[['Accesion_ID','Patient_ID','Morphology_ID','Organism_ID','Genus_ID','SpecType','Ward_ID']].notnull().all(1)]
    df_work = df_work.query('SpecType != "Unspecified" and Morphology_ID != "Unspecified"')
    df_work = df_work.query('AB_Class != "Unspecified" and AB_Name != "Unspecified"')
    df_work = df_work.query('SIR != ("X", "N")')
    df_work = df_work[df_work['SIR'].notnull()]
    df_work['Organism_ID'] = df_work['Organism_ID'].replace({'Acinetobacter baumannii/calcoaceticus complex' : 'Acinetobacter baumannii'})
    return  df_work

df_clean = clean(d_frame=df)
df_clean.describe()

,Patient_ID,Patient_Name,Accesion_ID,Date,AB_Class,Organism_ID,AB_Name,Morphology_ID,Genus_ID,SIR,MIC,RM,Month,Ward_ID,SpecType
count,67401,67103,67401,67401,67401,67401,67401,67401,67401,67401,64455,43859,67401,67401,67401
unique,1741,1649,2937,830,11,131,34,3,42,3,45,11,3,73,42
top,LE46628,INAYATUL MUSTAQH,1703161758C,3/9/2013 10:02,Beta-lactams,Escherichia coli,Ampicillin,Gram - Bacilli,Staphylococcus,R,>16,Extended Spectrum Beta-lactamase,2013/03,IRJ / Instalasi Rawat Jalan,Urine
freq,521,521,85,442,34486,14005,3724,41812,22438,39309,8690,13264,24946,5146,12214


From Our Simple Count It's look like Gram Negative are dominating all population


In [3]:
organism_count = df_clean[['Accesion_ID', 'Morphology_ID','Genus_ID','Organism_ID']].groupby(['Morphology_ID','Genus_ID','Organism_ID']).size().reset_index()
organism_count = organism_count.rename(columns={0 : 'size'}).sort_values(by=['Morphology_ID','size'], ascending=False)
organism_count.head(10)

,Morphology_ID,Genus_ID,Organism_ID,size
87,Gram - Bacilli,Escherichia,Escherichia coli,14005
94,Gram - Bacilli,Klebsiella,Klebsiella pneumoniae,8640
64,Gram - Bacilli,Acinetobacter,Acinetobacter baumannii,6159
108,Gram - Bacilli,Pseudomonas,Pseudomonas aeruginosa,5303
86,Gram - Bacilli,Enterobacter,Enterobacter cloacae,1855
129,Gram - Bacilli,Stenotrophomonas,Stenotrophomonas maltophilia,698
103,Gram - Bacilli,Proteus,Proteus mirabilis,546
124,Gram - Bacilli,Serratia,Serratia marcescens,463
72,Gram - Bacilli,Burkholderia,Burkholderia cepacia complex,340
85,Gram - Bacilli,Enterobacter,Enterobacter aerogenes,333


In [4]:
from bokeh.io import output_notebook, show
from bokeh.palettes import brewer
from bokeh.layouts import row
from bokeh.charts.attributes import  CatAttr
from bokeh.models.glyphs import HBar
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
output_notebook()

from bokeh.charts import Histogram, Bar

Loading BokehJS ...

In [14]:
#hist = Histogram(organism_count, values='size',color='orange', title = 'Organism Count')
#show(hist)
count_gram_positive = organism_count.query('Morphology_ID == ("Gram + Bacilli", "Gram + Cocci")').sort_values(by=['size'],  ascending=False)
count_gram_negative = organism_count.query('Morphology_ID != ("Gram + Bacilli", "Gram + Cocci")').sort_values(by=['size'],  ascending=False)

grams = ['positive','negative']

for gram in grams:
    labels = ('label_'+(gram))
    count = eval('count_gram_'+(gram))
    labels = CatAttr(df=count, columns='Organism_ID', sort = False)
    hist = ('hist_'+(gram))
    hist = Bar(count[:10], 
                    plot_height = 600, plot_width=470, label=labels, 
                    values='size', title='Most 10 Organism Gram '+gram, legend=None)
    show(hist)
print(count_gram_positive[:10])
print('__________________________________________')
print(count_gram_negative[:10])


   Morphology_ID        Genus_ID                            Organism_ID  size
13  Gram + Cocci  Staphylococcus                  Staphylococcus aureus  6849
23  Gram + Cocci  Staphylococcus            Staphylococcus haemolyticus  5273
24  Gram + Cocci  Staphylococcus                 Staphylococcus hominis  3589
20  Gram + Cocci  Staphylococcus             Staphylococcus epidermidis  3579
5   Gram + Cocci    Enterococcus                  Enterococcus faecalis  2607
38  Gram + Cocci  Staphylococcus                 Staphylococcus xylosus   602
33  Gram + Cocci  Staphylococcus              Staphylococcus schleiferi   310
15  Gram + Cocci  Staphylococcus                 Staphylococcus capitis   225
19  Gram + Cocci  Staphylococcus  Staphylococcus cohnii ssp urealyticum   224
34  Gram + Cocci  Staphylococcus                  Staphylococcus sciuri   224
__________________________________________
      Morphology_ID          Genus_ID                   Organism_ID   size
87   Gram - Bacilli     